<a href="https://colab.research.google.com/github/vikpy/DLSem3/blob/master/M1_Test_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyforest 
from pyforest import * 

  Created wheel for pyforest: filename=pyforest-1.0.3-py2.py3-none-any.whl size=13715 sha256=1b18f3a69ced626a13d7cf45a6a2a4a5f297aca1331d05db6cbd3317abb3765a
  Stored in directory: /root/.cache/pip/wheels/af/0b/39/340a7f15fc8d4ff5ab50847b28789afea04452a9d51b9721b5
Successfully built pyforest


# Q.1 Download three classes of data from ImageNet with wnid=n02111277, n02110341 and n02109961. Organize data into train , test and validate sets with 100, 50, 50 images respectively(2 Marks)

Ans:

In [ ]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import cv2
import PIL.Image
import urllib
import os
import pandas as pd



wnids = {
         
         'class1' : 'n02111277',
         
         'class2' : 'n02110341',

         'class3' : 'n02109961'

         }
image_net_path = 'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid='
pages = {}
for wnid in wnids.keys():
  temp = requests.get(image_net_path + wnids[wnid])
  pages[wnid] = str(BeautifulSoup(temp.content, 'html.parser')).split('\r\n')


In [ ]:
len(pages["class1"]), len(pages["class2"]), len(pages["class3"])

(1715, 1400, 1541)

In [ ]:
# Making directories for the train set 
!mkdir /content/train 
!mkdir /content/train/class1 
!mkdir /content/train/class2 
!mkdir /content/train/class3

# making directories for the test set
!mkdir /content/validation 
!mkdir /content/validation/class1 
!mkdir /content/validation/class2 
!mkdir /content/validation/class3 

# making directories for the test set 
!mkdir /content/test/ 
!mkdir /content/test/test 

In [ ]:
def url_to_image(url):
  import numpy as np
  import cv2
  import urllib
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype="uint8")
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
	# return the image
  return image

In [ ]:
# Creating the training examples 


n_of_training_images=100       #the number of training images to use

for class_ in pages:
  for progress in range(n_of_training_images):
      if(progress%20==0):
          print(progress)
      if not pages[class_][progress] == None:
        try:
          I = url_to_image(pages[class_][progress])
          if (len(I.shape)) == 3: #check if the image has width, length and channels
            save_path = '/content/train/' + class_ + '/img'+str(progress)+'.jpg'#create a name of each image
            cv2.imwrite(save_path,I)
        except:
          None

0
20
40
60
80
0
20
40
60
80
0
20
40
60
80


In [ ]:
# Creating the validation examples 

for class_ in pages:
  for progress in range(50):
      if(progress%20==0):
          print(progress)
      if not pages[class_][n_of_training_images + progress] == None:
        try:
          I = url_to_image(pages[class_][n_of_training_images + progress])
          if (len(I.shape)) == 3: #check if the image has width, length and channels
            save_path = '/content/validation/' + class_ + '/img'+str(n_of_training_images + progress)+'.jpg'#create a name of each image
            cv2.imwrite(save_path,I)
        except:
          None

0
20
40
0
20
40
0
20
40


In [ ]:
for class_ in pages:
  for progress in range(50):
      if(progress%20==0):
          print(progress)
      if not pages[class_][n_of_training_images + progress + 50] == None:
        try:
          I = url_to_image(pages[class_][n_of_training_images + progress + 50])
          if (len(I.shape)) == 3: #check if the image has width, length and channels
            save_path = '/content/test/test/' + str(class_) +  '_' +str(n_of_training_images + progress + 50) +'.jpg'#create a name of each image
            cv2.imwrite(save_path,I)
        except:
          None

0
20
40
0
20
40
0
20
40


# Q.2 Build a classification model for image recognition for images in Q1(6 Marks)

Ans:

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


trainval_datagen = ImageDataGenerator(
    rescale=1./255, horizontal_flip=True
)


test_datagen = ImageDataGenerator(
    rescale=1./255
)

#

#flow_from_directory()

train_generator = trainval_datagen.flow_from_directory(directory='/content/train/',
                                                       seed=7,
                                                       shuffle=True,
                                                       color_mode='rgb',
                                                       class_mode='categorical',
                                                       target_size=(150,150))

val_generator = trainval_datagen.flow_from_directory(directory='/content/validation/',                                                      
                                                       seed=7,
                                                       shuffle=True,                                                       
                                                       color_mode='rgb',
                                                       class_mode='categorical',
                                                       target_size=(150,150))



test_generator = test_datagen.flow_from_directory(directory='/content/test/',
                                                  seed=7,
                                                  shuffle=False,                                                  
                                                  class_mode=None,
                                                  color_mode='rgb',
                                                  target_size=(150,150))

model = Sequential()
model.add(Conv2D(512, (150,150), padding='valid', activation='relu', input_shape=(150,150,3)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Found 219 images belonging to 3 classes.
Found 95 images belonging to 3 classes.
Found 100 images belonging to 1 classes.
